In [359]:
import pandas as pd

In [360]:
df = pd.read_csv("player_data_sample.csv")

df = df.iloc[:-2]



In [361]:
df = df[df["Mins"] != "-"]

# Remove commas from Mins and convert to float
df["Mins"] = df["Mins"].str.replace(',', '').astype(float)

# Apply filters
df = df[df["Mins"] > 500]


# exclude players in youth leagues
df = df[~(df["Club"].str.contains("II| B$| C$", na=False))]

In [362]:
positions = sorted(df["Media Description"].unique(), 
                  key=lambda x: x.split()[-1] if isinstance(x, str) and x else '')

positions

['Centre-back',
 'Forward',
 'Full-back',
 'Goalkeeper',
 'Midfielder',
 'Striker',
 'Winger',
 'Attacking full back',
 'Veteran centre-back',
 'Enthusiastic centre-back',
 'Powerful centre-back',
 'Experienced centre-back',
 'Towering centre-back',
 'Commanding centre-back',
 'Veteran forward',
 'Veteran full-back',
 'Experienced full-back',
 'Strong full-back',
 'Young full-back',
 'Experienced goalkeeper',
 'Veteran goalkeeper',
 'Commanding goalkeeper',
 'Well-travelled goalkeeper',
 'Strong goalkeeper',
 'Instinctive goalkeeper',
 'Nomadic goalkeeper',
 'Towering goalkeeper',
 'Young goalkeeper',
 'Experienced midfielder',
 'Veteran midfielder',
 'Defensive midfielder',
 'Experienced attacking midfielder',
 'Attacking midfielder',
 'Veteran attacking midfielder',
 'Young midfielder',
 'Hard working midfielder',
 'Tenacious defensive midfielder',
 'Tireless midfielder',
 'Cultured midfielder',
 'Tireless defensive midfielder',
 'Well-travelled midfielder',
 'Tenacious midfielder',


In [363]:
pos_mapping = {
    "GK": "Goalkeeper",
    "LB": ["full back", "full-back"],
    "RB": ["full back", "full-back"],
    "CCB": "centre-back",
    "LCB": "centre-back",
    "RCB": "centre-back",
    "DM": "defensive midfielder",
    "CM": ["midfielder", "orchestrator"],
    "CAM": ["midfielder", "orchestrator"],
    "AM": ["attacking midfielder",  "orchestrator"],
    "LW": "winger",
    "RW": "winger",
    "LIF": ["winger", "attacking midfielder"],
    "RIF": ["winger", "attacking midfielder"],
    "DLF": ["striker", "forward"],
    "AF": ["striker", "forward"]
}

def find_players_by_position(df, position, pos_mapping):
    terms = pos_mapping[position]
    # Handle both single string and list of strings
    if isinstance(terms, str):
        terms = [terms]
    # Create regex pattern
    pattern = '|'.join(terms)
    return df[df["Media Description"].str.contains(pattern, case=False, na=False)]

# Usage:
am_players = find_players_by_position(df, "RCB", pos_mapping)

am_players

,Name,Position,Age,Nat,Height,Weight,Personality,Club,Transfer Value,Av Rat,...,OP-Cr %,OP-Crs A/90,OP-Crs C/90,Ps A/90,Shts Blckd/90,Shots Outside Box/90,xG/90,Saves/90,Inf,Rec
196,Aitor Córdoba,"D (C), DM",29.0,ESP,"5'11""",73 kg,Balanced,Burgos,£325K - £3.2M,6.93,...,0%,0.11,0,50.89,0.67,0,0,-,NaN,- - -
234,Anthony Briançon,"D (C), DM, M (C)",29.0,FRA,"6'1""",79 kg,Fairly Loyal,ASSE,£325K - £3.2M,6.82,...,0%,0.11,0,56.41,0.88,0,0,-,NaN,- - -
252,Naldo,D (C),36.0,BRA,"6'2""",82 kg,Balanced,Ferrol,£1K - £10K,6.71,...,0%,0.14,0,55.5,0.95,0,0,-,NaN,- - -
317,Enzo Ebosse,"D (LC), WB (L)",25.0,CMR,"6'1""",77 kg,Balanced,Udinese,£450K - £4.5M,6.75,...,-,0,0,55.31,0.81,0,0,-,NaN,- - -
380,Ousmane Camara,"D (C), DM",21.0,FRA,"6'6""",90 kg,Balanced,Angers SCO,£1.4M - £2.9M,6.78,...,0%,0.23,0,54.38,0.34,0,0,-,NaN,- - -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4872,Raphael Silva,D (C),32.0,BRA,"6'4""",82 kg,Fairly Professional,Esteghlal,£22K - £220K,6.95,...,-,0,0,20.71,0,0,0.01,-,NaN,- - -
4878,Seydine N'Diaye,"D (C), DM",26.0,FRA,"6'9""",87 kg,Balanced,BG Pathum Utd,£16K - £170K,7.01,...,-,0,0,20,0,0,0.08,-,NaN,- - -
4895,Jesper Tolinsson,D (C),21.0,SWE,"6'3""",83 kg,Balanced,Lommel SK,£170K - £1.7M,6.84,...,-,0,0,20,0,0,0.02,-,Wnt,- - -
4897,Charleston,D (LC),28.0,BRA,"6'4""",87 kg,Balanced,Jetisu,£4K - £12K,6.56,...,-,0,0,20,0,0,0.04,-,NaN,- - -


In [364]:
def transform_metrics(df):
    df_processed = df.copy()
    
    # Convert Mins to numeric first
    df_processed["Mins"] = pd.to_numeric(df_processed["Mins"], errors='coerce')
    
    # create metrics
    df_processed["Sv-xSv %"] = (pd.to_numeric(df_processed["Sv %"].str.rstrip('%'), errors='coerce') - 
                               pd.to_numeric(df_processed["xSv %"].str.rstrip('%'), errors='coerce'))
    
    # Handle Dist/90 - remove 'km' and convert to numeric
    if "Dist/90" in df.columns:
        df_processed["Dist/90"] = pd.to_numeric(df_processed["Dist/90"].str.rstrip('km'),errors='coerce')

    # Per 90 conversions
    for metric in ["Fls", "CCC", "Pres A", "Pres C"]:
        if metric in df.columns:
            df_processed[metric] = pd.to_numeric(df_processed[metric], errors='coerce')
            df_processed[metric] = (df_processed[metric] / df_processed["Mins"]) * 90
    
    # Inverse metrics (multiply by -1 so lower values become higher)
    for metric in ["Poss Lost/90", "Fls", "Gl Mst"]:
        if metric in df.columns:
            df_processed[metric] = pd.to_numeric(df_processed[metric], errors='coerce')
            df_processed[metric] = -1 * df_processed[metric]

            
    return df_processed

import numpy as np
df = transform_metrics(df)

In [365]:
goalkeeper = {
    # shot stopping - 0.6
    "Sv-xSv %": 0.25,
    "xGP": 0.35,
    # distribution - 0.3
    "Ps C/90": 0.05,
    "Pas %":0.05,
    "Pr passes/90": 0.1,
    "Gl Mst": 0.1,
    # aggressive claims - 0.1
    "Poss Won/90": 0.1
}

full_back = {
    # stamina / energy - 0.3
    "Dist/90": 0.15,
    "Sprints/90": 0.15,
    # progression - 0.2
    "Pr passes/90": 0.15,
    "Drb/90": 0.05,
    # crossing - 0.2
    "OP-Crs A/90": 0.1,
    "OP-Crs C/90": 0.1,
    # ball winner - 0.3
    "Tck/90": 0.08,
    "Int/90": 0.08,
    "Poss Won/90": 0.14
}

centre_centreback = {
    # aerial threat - 0.5
    "Hdrs W/90": 0.1,
    "Hdr %" : 0.15,
    "Aer A/90": 0.1,
    "K Hdrs/90": 0.15,
    # good defender / aggressive - 0.5
    "Tck R": 0.1,
    "K Tck/90": 0.15,
    "Tck/90": 0.05, # low weight
    "Int/90": 0.05, # low weight  
    "Clr/90": 0.05, # low weight
    "Blk/90": 0.05, # low weight
    "Poss Won/90": 0.05 # low weight
}

wide_centreback = {
    # aerial threat - 0.5
    "Hdrs W/90": 0.1,
    "Hdr %": 0.15,
    "Aer A/90": 0.1,
    "K Hdrs/90": 0.15,
    # progressive - 0.3
    "Pr passes/90": 0.2,
    "Drb/90": 0.1,
    # ball winner - 0.2
    "Tck/90": 0.07,
    "Int/90": 0.07,
    "Poss Won/90": 0.06
}

defensive_midfielder = {
    # intelligent - 0.25
    "Int/90": 0.1,
    "Poss Lost/90": 0.1, 
    "Fls": 0.05, 
    # ball winner - 0.4
    "Tck/90": 0.15,
    "Poss Won/90": 0.15,
    "K Tck/90": 0.1,
    # progressive - 0.35
    "Pr passes/90": 0.2,
    "Pas %": 0.1,
    "Ps A/90": 0.05
}

central_midfielder = {
    # stamina / energy - 0.2
    "Dist/90": 0.1,
    "Sprints/90": 0.1,
    # ball winner - 0.4
    "Tck/90": 0.1,
    "Int/90": 0.1,
    "Poss Won/90": 0.2,
    # progressive - 0.4
    "Pr passes/90": 0.25,
    "Drb/90": 0.1,
    "Pas %": 0.05
}

central_attacking_midfielder = {
    # stamina / energy - 0.2
    "Dist/90": 0.1,
    "Sprints/90": 0.1,
    # progressive  - 0.3 
    "Pr passes/90": 0.2,
    "Drb/90": 0.1,
    # goal threat - 0.3
    "Shots Outside Box/90": 0.06,
    "Shot/90": 0.12,
    "NP-xG/90": 0.12,
    # creative - 0.2
    "K Ps/90": 0.05,
    "xA/90": 0.05,
    "Ch C/90": 0.05,
    "CCC": 0.05 
}

attacking_midfielder = {
    # creative - 0.4
    "K Ps/90": 0.1,
    "xA/90": 0.1,
    "Ch C/90": 0.1,
    "CCC": 0.1,
    # dangerous dribbler - 0.2
    "Drb/90": 0.2,
    # goal threat - 0.25
    "Shot/90": 0.125,
    "NP-xG/90": 0.125,
    # presser - 0.15
    "Pres A": 0.075,
    "Pres C": 0.075
}

winger = {
    # crosser - 0.3
    "OP-Crs A/90": 0.1,
    "OP-Crs C/90":0.1,
    "xA/90": 0.1,
    # dangerous dribbler - 0.35
    "Drb/90": 0.4,
    # fast - 0.25
    "Sprints/90": 0.2,
    # presser - 0.1
    "Pres A": 0.05,
    "Pres C": 0.05
}

inside_forward = {
    # fast - 0.2
    "Sprints/90": 0.2,
    # dangerous dribbler - 0.3
    "Drb/90": 0.3,
    # goal threat - 0.25
    "Shot/90": 0.125,
    "NP-xG/90": 0.125,
    # presser - 0.1
    "Pres A": 0.05,
    "Pres C": 0.05,
    # creative - 0.15
    "xA/90": 0.15
}

deep_lying_forward = {
    # goal threat - 0.6
    "Shot/90": 0.3,
    "NP-xG/90": 0.3,
    # presser - 0.1
    "Pres A": 0.05,
    "Pres C": 0.05,
    # aerial threat - 0.2
    "Hdrs W/90": 0.06,
    "Hdr %": 0.07,
    "Aer A/90": 0.07,
    # good link up - 0.1
    "Ps C/90": 0.05,
    "Ch C/90": 0.05
}

advanced_forward = {
    # goal threat - 0.7
    "Shot/90": 0.35,
    "NP-xG/90": 0.35,
    # presser - 0.15
    "Pres A": 0.075,
    "Pres C": 0.075,
    # fast - 0.15
    "Sprints/90": 0.5
}

In [366]:
df["Wage"] = df["Wage"].str.replace('£', '')\
                      .str.replace('p/w', '')\
                      .str.replace(',', '')\
                      .str.strip()\
                      .astype(float)

league_values = df.groupby("Division")["Wage"].mean()
# Normalize to 0.6-1 range
league_quality = (league_values - league_values.min()) / (league_values.max() - league_values.min()) * 0.1 + 0.9

league_quality.sort_values(ascending=False).head(20)

Division
English Premier Division         1.000000
Saudi Professional League        0.986267
Qatar Stars League               0.975387
Saudi Second Division Group B    0.956755
Saudi Second Division Group A    0.956646
Italian Serie A                  0.945793
Bundesliga                       0.945749
Chinese Super League             0.943258
Saudi First Division League      0.942445
Major League Soccer              0.941329
UAE Professional League          0.941323
Qatar League                     0.936857
Russian Premier League           0.934872
Sky Bet Championship             0.931814
Turkish Super League             0.929596
Greek Super League 1             0.928207
Jupiler Pro League               0.927397
UAE First Division               0.926948
Argentine Premier Division       0.926182
Ligue 1 Uber Eats                0.926108
Name: Wage, dtype: float64

In [367]:
def calculate_pos_score(df, position_filter, position_metrics, league_quality, preferred_foot=None):
    pos_df = df[df["Media Description"].str.contains(position_filter, case=False, na=False)].copy()
    
    # Apply foot preference filter if specified
    if preferred_foot:
        if preferred_foot.lower() == 'left':
            pos_df = pos_df[pos_df["Preferred Foot"].str.contains("Left", na=False)]
        elif preferred_foot.lower() == 'right':
            pos_df = pos_df[pos_df["Preferred Foot"].str.contains("Right", na=False)]
    
    # Normalize metrics and apply weights
    for metric in position_metrics:
        if metric in pos_df.columns:
            if pos_df[metric].dtype == 'object':
                if pos_df[metric].str.contains('%').any():
                    pos_df[metric] = pd.to_numeric(pos_df[metric].str.rstrip('%'), errors='coerce')
                else:
                    pos_df[metric] = pd.to_numeric(pos_df[metric], errors='coerce')
            
            pos_df[f"{metric}_norm"] = (pos_df[metric] - pos_df[metric].min()) / \
                                     (pos_df[metric].max() - pos_df[metric].min())
    
    # Calculate base score
    total_metrics = len(position_metrics)
    weight = 1.0 / total_metrics
    pos_df["base_score"] = sum(pos_df[f"{metric}_norm"] * weight 
                              for metric in position_metrics)
    
    # Apply league quality multiplier
    pos_df["league_multiplier"] = pos_df["Division"].map(league_quality)
    pos_df["score"] = pos_df["base_score"] * pos_df["league_multiplier"]
    
    return pos_df.sort_values("score", ascending=False)

# Now this should work with your original goalkeeper list
players = calculate_pos_score(df, "defensive midfielder", defensive_midfielder, league_quality, preferred_foot=None)

players = players[players["Age"] <= 23]

players.head(10)

,Name,Position,Age,Nat,Height,Weight,Personality,Club,Transfer Value,Av Rat,...,Fls_norm,Tck/90_norm,Poss Won/90_norm,K Tck/90_norm,Pr passes/90_norm,Pas %_norm,Ps A/90_norm,base_score,league_multiplier,score
1242,Eric Martel,"D (C), DM, M (C)",22.0,GER,"6'2""",81 kg,Resolute,1. FC Köln,£4.9M - £6.2M,7,...,0.510021,0.709571,0.6290,0.648649,0.535050,0.851852,0.746021,0.677345,0.915928,0.620399
1606,Mattes Hansen,"DM, M (C)",20.0,GER,"6'0""",77 kg,Fairly Professional,Paderborn,£250K - £2.6M,7.09,...,0.814256,0.405941,0.4860,0.567568,0.805436,0.777778,0.806958,0.646445,0.915928,0.592097
987,William Clem,"DM, M (C)",20.0,DEN,"6'1""",70 kg,Fairly Professional,FC København,£375K - £3.7M,7.07,...,0.831588,0.224422,0.6870,0.675676,0.729614,0.740741,0.744202,0.637231,0.911983,0.581144
734,Cameron Humphreys,"DM, M (LC)",21.0,ENG,"5'10""",61 kg,Fairly Professional,Wycombe,£350K - £3.5M,7.03,...,0.768143,0.297030,0.7760,0.162162,0.937053,0.777778,0.754206,0.639343,0.906042,0.579272
4478,Niklas Tauer,"D (C), DM, M (C)",23.0,GER,"6'0""",72 kg,Fairly Determined,Braunschweig,£200K - £2M,6.96,...,0.496399,0.435644,0.6390,0.702703,0.702432,0.814815,0.793088,0.626723,0.915928,0.574033
1649,Ebrima Darboe,"DM, M (C)",23.0,GAM,"5'9""",69 kg,Fairly Ambitious,Frosinone,£26K - £250K,6.94,...,0.596512,0.669967,0.4665,0.243243,0.733906,0.851852,0.722146,0.622234,0.916101,0.570029
949,Jacob Christensen,"DM, M (C)",23.0,DEN,"5'11""",69 kg,Balanced,1. FC Köln,£2.1M - £4.2M,6.91,...,0.498897,0.389439,0.7005,0.351351,0.552217,0.851852,0.738063,0.620618,0.915928,0.568442
906,Martinelli,"DM, M (C)",23.0,BRA,"5'9""",70 kg,Balanced,FLU,£7.4M - £9.4M,6.99,...,0.783561,0.389439,0.7090,0.270270,0.579399,0.814815,0.682128,0.611194,0.914937,0.559205
1320,Joris Chotard,"DM, M (C)",23.0,FRA,"5'10""",70 kg,Fairly Professional,Montpellier,£6M - £7.4M,6.73,...,0.428234,0.567657,0.5995,0.324324,0.761087,0.888889,0.832651,0.603411,0.926108,0.558824
1277,George Thomason,"DM, M (C)",23.0,ENG,"5'10""",75 kg,Fairly Professional,Bolton,£1.1M - £2.4M,6.96,...,0.723924,0.425743,0.6195,0.297297,0.652361,0.888889,0.659391,0.581086,0.906042,0.526488
